# PEC Monte Carlo simulation

Start by importing MagmaPEC and MagmaPandas and any other packages you want to use. Here we also import Pandas for importing pressure data. For details on the use of MagmaPandas, please see it's [documentation](https://magmapandas.readthedocs.io/en/latest/).

In [1]:
import MagmaPEC as mpc
import MagmaPandas as mp

import pandas as pd

In the next few steps we import all relevant data and set up the melt initial FeO prediction model. These steps are identical to the [PEC model example](https://magmapec.readthedocs.io/en/latest/notebooks/pec_corr.html#)

Import melt inclusion and olivine data:

In [2]:
melt_file = "./data/melt.csv"
olivine_file = "./data/olivine.csv"

melt = mp.read_melt(melt_file, index_col=["name"])
olivine = mp.read_olivine(olivine_file, index_col=["name"])

Import inclusion internal pressures:

In [3]:
pressure_file ="./data/pressure.csv"
pressure = pd.read_csv(pressure_file, index_col = ["name"]).squeeze()

Set up the melt initial FeO prediction model:

In [4]:
wholerock_file = "./data/wholerock.csv"
wholerock = mp.read_melt(wholerock_file, index_col=["name"])

x = wholerock.drop(columns=["FeO"])
FeOi_predict = mpc.FeOi_prediction(x=x, FeO=wholerock["FeO"])

do_not_use = ["MnO", "P2O5", "Cr2O3", "total"]

model_fits = FeOi_predict.calculate_model_fits(exclude=do_not_use)
FeOi_predict.select_predictors(idx=3)

Next, we need to set up the object that handles the random sampling of errors in the Monte Carlo simulation. This is done with the *PEC_MC_parameters* class and it includes the following parameters for error propagation:

- **melt_errors**
        
    propagate errors on melt composition by providing one standard deviation errors per element as a pandas Series (fixed errors for all inclusions) or DataFrame (errors per inclusion).

- **olivine_errors**

    propagate errors on olivine composition by providing one standard deviation errors per element as a pandas Series (fixed errors for all inclusions) or DataFrame (errors per inclusion).

- **FeOi_errors**

    propagate errors on estimate melt initial FeO contents. Fixed errors can be provided either for the whole dataset, or per inclusion. Alternatively, an [FeOi_prediction object](https://magmapec.readthedocs.io/en/latest/notebooks/FeOi.html) can be provided to propagate errors on predictions models.

- **Fe3Fe2**

    Propagate errors on modelled melt Fe<sup>2+</sup>/Fe<sup>3+</sup> ratios. Errors are automatically calculated by MagmaPandas based on the selected model. Pass *True* to this parameter to activate it.

- **Kd**

    Propagate errors on modelled olivine-melt Fe-Mg partition coefficients. Errors are automatically calculated by MagmaPandas based on the selected model. Pass *True* to this parameter to activate it.

By default errors are not propagated - you explicitely need to tell MagmaPEC to do so when initialising the *PEC_MC_parameters* object

In this example we will use all error propagation options, which means we need to provide melt and olivine composition errors. We import these from .csv files containing error data for individual inclusions and olivines. This is just an example with randomly generated errors, normally you should use analytical errors.

In [5]:
melt_errors_file = "./data/melt_errors.csv"
olivine_errors_file = "./data/olivine_errors.csv"

melt_errors = pd.read_csv(melt_errors_file, index_col=[0])
olivine_errors = pd.read_csv(olivine_errors_file, index_col=[0])

Make very sure that the elements in the error data have identical sorting to the melt and olivine dataframes, otherwise errors will be applied to the wrong elements. 
We can force this by sorting the columns of the error dataframes (or series) with the *elements* attributes of the melt and olivine MagmaFrames:

In [6]:
melt_errors = melt_errors[melt.elements]
olivine_errors = olivine_errors[olivine.elements]

Here's what they look like:

In [7]:
melt_errors.head()

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl
PI032-04-01,1.02,0.64,0.14,0.47,0.69,0.10,0.08,0.05,0.18,0.05,0.05,0.17,0.21,0.22,0.10,0.12
PI032-04-02,1.06,0.84,0.29,0.46,0.54,0.05,0.03,0.01,0.06,0.18,0.04,0.24,0.12,0.13,0.12,0.04
PI041-02-02,1.04,0.90,0.09,0.55,0.49,0.22,0.21,0.01,0.16,0.04,0.00,0.15,0.23,0.04,0.17,0.06
PI041-03-01,0.98,0.64,0.10,0.40,0.53,0.12,0.12,0.08,0.01,0.03,0.00,0.05,0.36,0.06,0.14,0.04
PI041-03-03,1.02,0.54,0.31,0.42,0.68,0.18,0.03,0.02,0.11,0.06,0.10,0.18,0.08,0.13,0.06,0.15


In [8]:
olivine_errors.head()

,SiO2,FeO,MgO,NiO,MnO,Al2O3,CaO
PI032-04-01,1.06,0.60,1.22,0.01,0.09,0.08,0.02
PI032-04-02,0.85,0.57,1.28,0.14,0.07,0.24,0.06
PI041-02-02,1.11,0.44,1.30,0.08,0.10,0.01,0.19
PI041-03-01,0.99,0.50,1.31,0.14,0.16,0.15,0.01
PI041-03-03,0.95,0.64,1.21,0.13,0.11,0.10,0.15


Together with the *FeOi_prediction* object, we pass these as arguments to the *PEC_MC_parameters* object. We also set *Fe3Fe2* and *Kd* to *True* in order to propagate their model errors.

In [9]:
mc_parameters = mpc.PEC_MC_parameters(melt_errors=melt_errors, olivine_errors=olivine_errors, FeOi_errors=FeOi_predict, Fe3Fe2=True, Kd=True, temperature=True)

Now we can create the Monte Carlo model with the *PEC_MC* object

In [10]:
pec_mc_model = mpc.PEC_MC(inclusions=melt, olivines=olivine, P_bar=pressure, FeO_target=FeOi_predict, MC_parameters=mc_parameters)

And finally run the Monte Carlo simulation with *n* iterations

In [12]:
pec_mc_model.run(n=40)

Monte Carlo loop
040/040
Equilibrating ... |██████████████████████████████| 100% [10/10] in 2.8s 
Correcting    ... |██████████████████████████████| 100% [10/10] in 11.8s 


Results are stored internally in the following attributes:

- **pec**: pandas DataFrame

    Average PEC extents (%) of the MC model and their one standard deviation errors.

- **inclusions_corr**: MagmaPandas Melt frame

    Averages of corrected melt inclusion compositions (wt. %)

- **inclusions_stddev**: pandas DataFrame

    One standard deviation errors on inclusions_corr (wt. %)

- **pec_MC**: pandas DataFrame

    PEC extents for individual iterations

- **inclusions_MC**: dictionary of MagmaPandas Melt frames

    corrected melt inclusion compositions for individual iterations.

In [13]:
pec = pec_mc_model.pec
inclusions_corrected = pec_mc_model.inclusions_corr
inclusions_errors = pec_mc_model.inclusions_stddev

pec_mc = pec_mc_model.pec_MC
inclusions_MC = pec_mc_model.inclusions_MC

In [14]:
pec

,pec,stddev
name,,
PI032-04-01,10.179825,4.589837
PI032-04-02,11.095848,3.690214
PI041-02-02,0.232676,3.069757
PI041-03-01,13.612856,4.473047
PI041-03-03,13.428424,4.850329
PI041-05-04,-3.987388,2.121582
PI041-05-06,1.984667,2.813967
PI041-07-01,13.081787,5.073188
PI041-07-02,11.36792,4.617962


In [15]:
inclusions_corrected

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl
name,,,,,,,,,,,,,,,,
PI032-04-01,48.922997,13.984594,7.599797,9.735582,10.27152,3.61715,0.66775,0.143549,2.451619,0.284739,-0.000001,0.629764,1.368817,0.127031,0.134006,0.061087
PI032-04-02,48.245122,14.824449,7.51047,9.458191,10.325269,3.465164,0.916781,0.139653,2.585731,0.307811,-0.000001,0.640394,1.298411,0.098215,0.139277,0.045062
PI041-02-02,49.134664,16.859113,4.867277,9.203424,10.193243,3.838824,1.057699,0.153101,2.774484,0.554537,0.0,0.46498,0.704823,0.057042,0.096273,0.040517
PI041-03-01,45.846473,15.623865,7.275567,10.920469,10.698974,3.336694,1.149272,0.133842,3.127595,0.556251,-0.000001,0.783695,0.332374,0.063219,0.098836,0.052875
PI041-03-03,45.356746,15.862556,7.074067,11.125737,10.748104,3.417292,1.157899,0.090563,3.228128,0.514783,-0.0,0.836945,0.317263,0.110262,0.076439,0.083217
PI041-05-04,47.823651,18.593467,3.722929,9.464024,9.385233,4.625283,1.606307,0.137177,2.509186,0.827986,-0.000001,0.517401,0.496083,0.109581,0.122847,0.058844
PI041-05-06,46.371568,17.137365,4.671482,8.995056,11.39395,3.914714,1.480397,0.174469,3.61477,0.629164,-0.000001,0.698057,0.597377,0.125922,0.130388,0.065322
PI041-07-01,45.675681,15.383933,7.301022,9.693416,11.518826,3.1342,1.268498,0.130712,3.521753,0.554281,-0.000002,0.45975,1.013496,0.116776,0.16898,0.058678
PI041-07-02,45.871928,15.608597,6.86786,10.062807,11.456399,3.236709,1.333192,0.133986,3.549983,0.61958,-0.000001,0.400043,0.56275,0.073844,0.148048,0.074276


In [16]:
inclusions_errors

,SiO2_stddev,Al2O3_stddev,MgO_stddev,CaO_stddev,FeO_stddev,Na2O_stddev,K2O_stddev,MnO_stddev,TiO2_stddev,P2O5_stddev,Cr2O3_stddev,CO2_stddev,H2O_stddev,F_stddev,S_stddev,Cl_stddev
name,,,,,,,,,,,,,,,,
PI032-04-01,0.665046,0.785207,1.318968,0.546537,0.351261,0.173916,0.073265,0.049494,0.155408,0.036534,0.000002,0.156441,0.170411,0.157885,0.074166,0.081596
PI032-04-02,0.597126,0.666215,1.086001,0.482435,0.250312,0.131086,0.040827,0.009021,0.083375,0.129305,0.000001,0.196497,0.124493,0.08733,0.103285,0.03543
PI041-02-02,0.85478,1.132328,0.926083,0.531579,0.391163,0.256938,0.19938,0.015193,0.1636,0.037862,0.000002,0.152972,0.246839,0.036035,0.124706,0.053307
PI041-03-01,0.649729,0.844777,1.238903,0.463405,0.152115,0.149901,0.103893,0.066605,0.11753,0.034858,0.000002,0.061315,0.252344,0.051373,0.097012,0.032789
PI041-03-03,0.61624,0.720897,1.307022,0.447835,0.261636,0.193315,0.044478,0.020464,0.100298,0.052126,0.000002,0.156626,0.06328,0.096598,0.050562,0.101822
PI041-05-04,0.818786,0.743469,0.704273,0.451134,0.241318,0.203144,0.160292,0.122182,0.079671,0.029617,0.000002,0.206004,0.186774,0.085434,0.010275,0.010391
PI041-05-06,0.86654,1.085729,0.789279,0.303002,0.372291,0.336916,0.233785,0.178582,0.148027,0.05603,0.000002,0.227059,0.314796,0.085845,0.053899,0.026356
PI041-07-01,0.605306,1.071266,1.274165,0.444165,0.223674,0.143167,0.050673,0.052658,0.12778,0.027716,0.000001,0.037803,0.172466,0.107581,0.027096,0.081821
PI041-07-02,0.569756,0.901254,1.295664,0.353091,0.182747,0.192634,0.12955,0.057103,0.127745,0.021417,0.000002,0.210702,0.193902,0.002484,0.048413,0.063839


In [17]:
pec_mc.head()

name,PI032-04-01,PI032-04-02,PI041-02-02,PI041-03-01,PI041-03-03,PI041-05-04,PI041-05-06,PI041-07-01,PI041-07-02,PI052-01-02
iteration,,,,,,,,,,
0,6.177443,9.598209,-2.539465,7.637873,8.4,-6.485112,-0.679897,8.978094,6.161604,-9.711079
1,20.177089,14.80899,5.353601,18.138152,18.264048,-2.043932,5.179742,18.606693,19.160636,-5.814
2,11.744748,15.580285,1.198683,20.599313,11.420996,-1.800607,2.977377,17.523083,16.109906,-7.628687
3,7.323833,9.777246,-2.872969,11.261172,7.283502,-6.978813,-2.415373,7.990621,8.606056,-9.01991
4,12.234181,10.095841,-0.504837,14.571003,18.516884,-1.743463,1.929218,16.633659,14.259519,-4.250232


The dataframes in *inclusions_MC* also have *isothermal_equilibration* and *Kd_equilibration* columns. These columns show if equilibration during stage [1] and [2] respectively was successful. In extreme cases, error propagation of melt and olivine compositions by random sampling can result in melt-olivine pairs that cannot be equilibrated without crystallising more than the entire inclusion or exchanging more Mg or Fe than is present in the inclusion. If that is the case, no corrected compositions are calculated and eiher the isothermal- of Kd-equilibration column is set to *False*.

In [18]:
inclusions_MC["PI032-04-01"].head()

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl,isothermal_equilibration,Kd_equilibration
iteration,,,,,,,,,,,,,,,,,,
0,48.292666,14.876233,6.282982,10.03151,10.360428,3.666842,0.693977,0.124677,2.630422,0.240964,-0.000001,0.755377,1.483278,0.330591,0.230053,-0.0,True,True
1,47.755594,13.708518,9.957488,8.624569,10.782005,3.217022,0.691082,0.137937,2.582962,0.250828,-0.000003,0.636978,1.336308,0.108633,0.148449,0.06163,True,True
2,48.964719,14.636007,8.03674,9.144718,10.079897,3.51701,0.648943,0.125437,2.330119,0.293616,-0.000003,0.799017,1.159119,-0.0,0.056881,0.207782,True,True
3,49.638647,14.420456,6.034834,10.577057,10.082005,3.583692,0.70313,0.152399,2.425394,0.236438,0.000002,0.72127,1.183168,0.118123,0.123386,0.0,True,True
4,48.252962,13.927559,8.731397,9.190609,10.211029,3.628077,0.771115,0.196084,2.37669,0.272963,-0.000004,0.546669,1.438668,0.449783,0.006401,-0.000001,True,True
